## Building an Audio GPT

based on notebook by AK [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Training Code:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 200
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('/content/drive/MyDrive/AI/amen-hexfile.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data)) # first 90% will be train, rest val
train_data = data[:]
val_data = data[:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
tk = (decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.206246 M parameters
step 0: train loss 3.7927, val loss 3.7964
step 100: train loss 3.1772, val loss 3.1795
step 199: train loss 2.9284, val loss 2.9308


In [ ]:
# context.shape

torch.Size([1, 1])

In [ ]:
max_iters = 10000

In [ ]:
learning_rate = 1e-3 # going from 3 to 7 seems to work

In [ ]:
# create a PyTorch optimizer
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
tk = (decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 0.1604, val loss 0.1603
step 100: train loss 0.1607, val loss 0.1594
step 200: train loss 0.1596, val loss 0.1601
step 300: train loss 0.1607, val loss 0.1611
step 400: train loss 0.1608, val loss 0.1593
step 500: train loss 0.1596, val loss 0.1601
step 600: train loss 0.1600, val loss 0.1598
step 700: train loss 0.1605, val loss 0.1603
step 800: train loss 0.1600, val loss 0.1601
step 900: train loss 0.1603, val loss 0.1597
step 1000: train loss 0.1598, val loss 0.1604
step 1100: train loss 0.1602, val loss 0.1598
step 1200: train loss 0.1599, val loss 0.1595
step 1300: train loss 0.1598, val loss 0.1596
step 1400: train loss 0.1596, val loss 0.1599
step 1500: train loss 0.1603, val loss 0.1597
step 1600: train loss 0.1598, val loss 0.1604


KeyboardInterrupt: ignored

In [ ]:
print(tk)

In [ ]:
torch.save(model, '/content/drive/MyDrive/AI/gpt2-audio-model-GREAT.pt')

In [ ]:
# tk = (decode(m.generate(context, max_new_tokens=1999)[0].cpu().numpy()))

In [ ]:
tk = (decode(m.generate(context, max_new_tokens=1999)[0].tolist()))

In [ ]:
# tkarr = np.array(tk)

parse list

In [ ]:
# tkl = (tk.split('\n')) 

In [ ]:
tka = np.array(tk.split('\n'))

In [ ]:
tka.shape

(224,)

In [ ]:
tka

array(['', '4v4as6f2', 'f21er21q', 'hk1cccnk', 'oj6d-cr9', '87nujfks',
       '2fj0hw86', 'jo54mqi5', '9bqnaaoc', 'ctcun4l9', '3tidmnd3',
       'rdrujqq4', 'et8b33n6', '9qosrg3g', '5ggeotkt', 'mtj3brhi',
       'm27er1-v', '5ekflle6', 'm2qnmli5', 'ctqceogs', 'hz34ljmq',
       '89g02de9', 'ifhaa3s3', 'ojl7bh63', '2fi9c5gn', '9bbibi9y',
       '9behbaq6', '1ujt-b7k', '5g1x2lk5', 'm2krrzeo', 'dke5eaiq',
       '7z2fgqol', 'm2qs5ir0', 'dkodeoci', 'n3h1e4r0', '4v642fec',
       'nnay6pc5', 'hkrb8k6s', '403ac6k2', '9b6hb6mt', 'ct5pa5fy',
       '18-47i5i', 'efcf5uii', 'craxb0ax', '-d4i-08m', '1abbaql1',
       'kphwpla1', '95n46qiw', 'grgrhh4w', 'j5maqzfb', 'm25upfbm',
       'e92u5g2r', '6amaf93f', 'bns7qzpc', '5tdo73hv', 'nxeerh5z',
       'o1ee10kl', 'o1ee10kl', '1qeelukl', '1qeeiukl', '1qeelukl',
       '1qeeiukl', '1qeeluip', 'b3cv3ank', 'qbfaeg9l', 'j03rpho3',
       'hk6si2f2', 'hkbg22e7', 'hk35aklm', 'hzl1a3qj', '2f6e7ocs',
       '4smj43s4', 'ltj37es2', '9b67bvs8', 'inlqhwl7', 'in

In [ ]:
import numpy as np

In [ ]:
tka.dtype

dtype('<U8')

In [ ]:
hexarr = tka

In [ ]:
# hexarr = tk.cpu().numpy().astype(np.int8)

In [ ]:
hexarr.shape

(224,)

In [ ]:
hexarr.dtype

dtype('<U8')

In [ ]:
hexarr

In [ ]:
!pip install base36
import base36

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dcarr = []
st = ''
for x in hexarr:
  for i, y in enumerate(x):
    m = i % 2 
    if (m == 0):
      st = st + y
    elif (m == 1):
      st = st + y
      if (st[1] == '-'):
        nb = base36.loads(st[0])
      else:
        nb = base36.loads(st)
      ni = int(nb)
      if (ni < 0):
        spl = st.split("-")[1]
        nb = base36.loads(spl)
      if (nb > 1023):
        nb = 0
        print("Didn't work, this shouldn't be over 1023 -->>", nb)  
      dcarr = np.append(dcarr, nb)
      # print(nb)
      st = ''

Didn't work, this shouldn't be over 1023 -->> 0


In [ ]:
dcarr.dtype

dtype('float64')

In [ ]:
ts = torch.from_numpy(dcarr.astype(np.int64))

In [ ]:
dn = 880 / 4

In [ ]:
dn

220.0

In [ ]:
tst = ts[:880]

In [ ]:
tst.shape

torch.Size([880])

In [ ]:
for x in tst:
  if (x > 1023):
    print (x)

In [ ]:
tsr = tst.reshape(220,4) # could be the known shape or len over 4 (or num quantisers)

In [ ]:
rst = tsr.T

In [ ]:
rst.shape

torch.Size([4, 220])

In [ ]:
# torch.equal(cs,rst)

In [ ]:
!pip install encodec

In [ ]:
from encodec import EncodecModel
from encodec.utils import convert_audio
from PIL import Image
import numpy as np
import torchaudio
import torch
from IPython.display import Audio
import torchvision.transforms as transforms  
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [ ]:
tn = rst
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(3.0)
uc = tn.unsqueeze(0) # unsqueeze codes # only needed if 2d array
sco = torch.split(uc, 1) # split codes
lis = list(sco) # turn to list
lis.append(None) # idk why, but it works I think
flis = [lis] # again don't ask too many questions

with torch.no_grad():
  decoded_frames = model.decode((flis))
Audio(decoded_frames.squeeze(),rate=24000)

In [ ]:
flis